In [171]:
import numpy as np

In [172]:

# trace ray with no reflection 
# different r should give the same value as long as the sensor plane is at the focus point for the lens
# different theta should change it linearly

# ideas: keep this part in python and integrate w/ cpp?

def T(d):
    return np.matrix([[1, d], [0, 1]])
    
def R(ri, n1, n2):
    return np.matrix([[1, 0], [(n1-n2)/(n2*ri), n1/n2]])
    
def L(ri):
    return np.matrix([[1, 0], [2/ri, 1]])

pupil_height=12.3

R1 = R(30.810, 1.000, 1.652)
T1 = T(7.700)
R2 = R(-89.350, 1.652, 1.603)
T2 = T(1.850)
R3 = R(580.380, 1.603, 1.000)
T3 = T(3.520)
R4 = R(-80.630, 1.000, 1.643)
T4 = T(1.850)
R5 = R(28.340, 1.643, 1.000)
T5 = T(4.180)

# now, we've reached the aperature plane
# height = 11.6
T6 = T(3.000)

R7 = np.matrix([[1, 0], [0, 1.000/1.581]])
T7 = T(1.850)
R8 = R(32.190, 1.581, 1.694)
T8 = T(7.270)
R9 = R(-52.990, 1.694, 1)
#T9 = T(81.857)
T9 = T(81.320)

def trace_ray(r, theta):
    r = np.array([r, theta])

    Ma = T5.dot(R5).dot(T4).dot(R4).dot(T3).dot(R3).dot(T2).dot(R2).dot(T1).dot(R1)
    Ms = T9.dot(R9).dot(T8).dot(R8).dot(T7).dot(R7).dot(T6)
    
    return Ms.dot(Ma).dot(r)
    
def trace_ray_with_ghost(r, theta):
    r = np.array([r, theta])
    
    L4 = L(1.643)
    L2 = L(1.603)
    R3_inv = np.linalg.inv(R3)
    L2_inv = np.linalg.inv(L2)
 

    M1 = T3.dot(R3).dot(T2).dot(R2).dot(T1).dot(R1)
    Bounce = T2.dot(L2_inv).dot(T2).dot(R3_inv).dot(T3).dot(L4)
    M2 = T5.dot(R5).dot(T4).dot(R4).dot(T3).dot(R3)
    Ms = T9.dot(R9).dot(T8).dot(R8).dot(T7).dot(R7).dot(T6)
    
    return Ms.dot(M2).dot(Bounce).dot(M1).dot(r)

def trace_ray_with_ghost_2(r, theta):
    r = np.array([r, theta])
    
    L4 = L(1.643)
    L1 = L(1.652)
    R3_inv = np.linalg.inv(R3)
    L1_inv = np.linalg.inv(L1)
    R2_inv = np.linalg.inv(R2)
 

    M1 = T3.dot(R3).dot(T2).dot(R2).dot(T1).dot(R1)
    Bounce = T2.dot(L1_inv).dot(T1).dot(R2_inv).dot(T2).dot(R3_inv).dot(T3).dot(L4)
    M2 = T5.dot(R5).dot(T4).dot(R4).dot(T3).dot(R3).dot(T2).dot(R3)
    Ms = T9.dot(R9).dot(T8).dot(R8).dot(T7).dot(R7).dot(T6)
    
    return Ms.dot(M2).dot(Bounce).dot(M1).dot(r)

In [173]:
# 0.17 is the max radians since we use small angle approximations
# r should we 12.3 since that's the height of the entrance pupil.


In [174]:
# varying r gives us the same thing approximately
print(trace_ray(12.3, 0.1))
print(trace_ray(1, 0.1))


[[ 9.92340672 -0.03497702]]
[[9.92396965 0.07889318]]


In [175]:
# varying theta changes r_out linearly
# theta_out shouldn't be the same
print(trace_ray(5, 0.5))
print(trace_ray(5, 0.1))

[[49.61984825  0.39446591]]
[[9.92377038 0.03858515]]


In [176]:
# changing radius should ghost linearly
print(trace_ray_with_ghost(0.5, 0.001))
print(trace_ray_with_ghost(1, 0.001))

[[-434.08824529   -4.14980023]]
[[-857.4118636    -8.19669154]]


In [177]:
# different ghosts should have different sizes
print(trace_ray_with_ghost(12.3, 0.001))
print(trace_ray_with_ghost_2(12.3, 0.001))

# but why so big??

[[-10424.52563746    -99.65643518]]
[[-32000.99989635   -302.18311581]]


In [178]:
# changing theta
print(trace_ray_with_ghost_2(12.3, 0.01))
print(trace_ray_with_ghost_2(12.3, 0.02))
print(trace_ray_with_ghost_2(12.3, 0.03))


# radius always gets bigger with bigger theta... 
# so if the sun is further away from the center, there's bigger lens flares? 
# not sure this is true

[[-32300.86635261   -305.01473834]]
[[-32634.05130402   -308.1609856 ]]
[[-32967.23625542   -311.30723285]]
